In [1]:
### Maximal Marginal Relevance
# MMR (Maximal Marginal Relevance) is a powerful diversity-aware 
# retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

c:\Users\hites\OneDrive\Desktop\AI\agentic-rag\hybrid-search-strategies\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [12]:
# Step 1: Load and chunk the document
loader = TextLoader("data/langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)

In [13]:
# Step 2: FAISS Vector Store with HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [14]:
### Step 3: Create MMR Retirever
retriever=vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3}
)

In [15]:
# Step 4: Prompt and LLM
prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.

Context:
{context}

Question: {input}
""")
llm=init_chat_model("groq:gemma2-9b-it")

In [16]:
# Step 5: RAG Pipeline
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [17]:
# Step 6: Query
query = {"input": "How does LangChain support agents and memory?"}
response = rag_chain.invoke(query)

print("✅ Answer:\n", response["answer"])

✅ Answer:
 LangChain supports agents by allowing them to utilize tools like calculators, search APIs, custom functions, and interact with external APIs and databases. This enhances their capabilities to perform complex tasks. 

LangChain supports memory through:

* **ConversationBufferMemory:**  Retains previous interactions for coherent multi-turn conversations.
* **ConversationSummaryMemory:**  Summarizes past conversations, providing a condensed overview for context. 


This combination enables agents to access relevant information from past interactions and utilize appropriate tools to effectively carry out tasks. 



In [18]:
response

{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='4d52cfa1-e0ca-49cc-bf62-67a9bfbe27da', metadata={'source': 'data/langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='36a93edc-1e1f-4abd-9c78-7dce1cc04825', metadata={'source': 'data/langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='73983f89-9b48-45cd-beff-4e9a5b79fba7', metadata={'source': 'data/langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in 